# Connect to Database

In [5]:
from query_database import QueryDatabase

df = QueryDatabase("SELECT * FROM supplier_invoice.invoice_202408;").df
# .df 表示查询结果被返回为一个 Pandas DataFrame.
df.head()


,Förvaltning,Leverantör,Organisationsnummer,Verifikationsnummer,Konto,Kontotext,Belopp exkl moms
0,Stadsmiljönämnden,TRACK TEC GMBH,106/5727/0626,4001291513,4101,Inköp anläggnings och underhållsmaterial,"9 835 315,00"
1,Kretslopp och Vatten,POLISMYNDIGHETEN I VÄSTRA GÖTALAND,2021000076,5601378982,6185,Anläggningsentreprenad,"870,00"
2,Kretslopp och Vatten,POLISMYNDIGHETEN I VÄSTRA GÖTALAND,2021000076,5601377374,6185,Anläggningsentreprenad,"870,00"
3,Kretslopp och Vatten,POLISMYNDIGHETEN I VÄSTRA GÖTALAND,2021000076,5601378519,6185,Anläggningsentreprenad,"870,00"
4,Exploateringsnämnden,POLISMYNDIGHETEN I VÄSTRA GÖTALAND,2021000076,2001226894,7641,Diverse skatter och offentliga avgifter,"1 000,00"


In [6]:
# show how much data
df.shape


(92989, 7)

## clean cost column

In [13]:
# 取 df 中 的 Belopp exkl moms 这一列， df 是 8月份表格的 DataFrame       straightforward 直截了当

df.loc[:, "Belopp exkl moms"] = df["Belopp exkl moms"].str[:-3].str.replace(" ", "").astype(int)

# .str[:-3] 去掉后3位, .str.replace(" ", "") 去掉空格, .astype(int) 格式从 object 变成 int
# object 类型 是 Pandas 中的通用数据类型，它通常用于存储文本数据（字符串），DataFrame 中的某一列包含混合类型的数据（如字符串、整数、浮动小数等）
# df.loc[:, "Belopp exkl moms"]：把新更改的值给 「选择所有的行, Belopp exkl moms 列」

df.head()

,Förvaltning,Leverantör,Organisationsnummer,Verifikationsnummer,Konto,Kontotext,Belopp exkl moms
0,Stadsmiljönämnden,TRACK TEC GMBH,106/5727/0626,4001291513,4101,Inköp anläggnings och underhållsmaterial,9835315
1,Kretslopp och Vatten,POLISMYNDIGHETEN I VÄSTRA GÖTALAND,2021000076,5601378982,6185,Anläggningsentreprenad,870
2,Kretslopp och Vatten,POLISMYNDIGHETEN I VÄSTRA GÖTALAND,2021000076,5601377374,6185,Anläggningsentreprenad,870
3,Kretslopp och Vatten,POLISMYNDIGHETEN I VÄSTRA GÖTALAND,2021000076,5601378519,6185,Anläggningsentreprenad,870
4,Exploateringsnämnden,POLISMYNDIGHETEN I VÄSTRA GÖTALAND,2021000076,2001226894,7641,Diverse skatter och offentliga avgifter,1000


In [15]:
from database2 import Database
from constants import DATABASE_PATH

with Database(DATABASE_PATH) as db:
    db.connection.register('df', df)   # 'df' is defined name 
    # 调用了 db.connection 对象的 register 方法，注册到数据库连接中。'df' 是名称，df 是数据表（Pandas DataFrame 对象）

    db.query("CREATE SCHEMA IF NOT EXISTS refined_layer;")
    db.query(f"""
             CREATE TABLE IF NOT EXISTS refined_layer.invoice_202408
             as select * from df;
             """)




In [17]:
QueryDatabase("select * from refined_layer.invoice_202408;").df.head()

,Förvaltning,Leverantör,Organisationsnummer,Verifikationsnummer,Konto,Kontotext,Belopp exkl moms
0,Stadsmiljönämnden,TRACK TEC GMBH,106/5727/0626,4001291513,4101,Inköp anläggnings och underhållsmaterial,9835315
1,Kretslopp och Vatten,POLISMYNDIGHETEN I VÄSTRA GÖTALAND,2021000076,5601378982,6185,Anläggningsentreprenad,870
2,Kretslopp och Vatten,POLISMYNDIGHETEN I VÄSTRA GÖTALAND,2021000076,5601377374,6185,Anläggningsentreprenad,870
3,Kretslopp och Vatten,POLISMYNDIGHETEN I VÄSTRA GÖTALAND,2021000076,5601378519,6185,Anläggningsentreprenad,870
4,Exploateringsnämnden,POLISMYNDIGHETEN I VÄSTRA GÖTALAND,2021000076,2001226894,7641,Diverse skatter och offentliga avgifter,1000


In [22]:
QueryDatabase("""
              select 
                Leverantör, 
                sum("Belopp exkl moms") as "Summa (SEK)"
              from 
                refined_layer.invoice_202408 
              group by 
                Leverantör
              order by
                "Summa (SEK)"
              desc limit 10; 
              """).df

,Leverantör,Summa (SEK)
0,GÖTEBORGS STADS LEASING AB,62433766.0
1,KPA PENSIONSSERVICE AB,59741750.0
2,RÄDDNINGSTJÄNSTEN STORGÖTEBORG,55320824.0
3,PURAC AB,48603208.0
4,RENOVA,34435107.0
5,VÄSTTRAFIK AB,28616573.0
6,MARK OCH ENERGIBYGGARNA,24775068.0
7,ATTENDO SVERIGE AB,23593435.0
8,MARTIN&SERVERA RESTAURANGHANDEL,23323653.0
9,GRYAAB AB,22961946.0
